In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm

import matplotlib.pyplot as plt


import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier

from sklearn import linear_model




/Users/goris/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
df_mp = pd.read_json(path_or_buf="file://localhost/Users/goris/nasa_nea/mpcorb_extended.json")


In [3]:
df_nea = pd.read_json(path_or_buf="file://localhost/Users/goris/nasa_nea/nea_extended.json")


In [4]:
df_mp_dups = pd.concat([df_mp, df_nea])

# Dropping duplicates based on columns that are highly probable to have unique values per-row
df_mp_pure = df_mp_dups.drop_duplicates(subset=['Peri', 'n'], keep=False)


In [5]:
df_pha = df_nea[df_nea['PHA_flag'] == 1]


In [6]:
df_nea_pure = df_nea[df_nea['PHA_flag'].isnull()]

In [7]:
print 'Master Set Count =', len(df_mp_pure.index)
print 'NEA Set Count =', len(df_nea_pure.index)
print 'PHA Set Count =', len(df_pha.index)

Master Set Count = 699486
NEA Set Count = 12418
PHA Set Count = 1679


In [8]:
#Adding Labels to the data
df_mp_pure['Label'] = 2
df_nea_pure['Label'] = 1
df_pha['Label'] = 0


/Users/goris/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/goris/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/goris/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [9]:

df_joined = pd.concat([df_mp_pure, df_nea_pure, df_pha])

In [10]:
#Deleting unrelevant features from the dataset
df_joined = df_joined.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
                            'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U', 'Arc_length'],axis=1)       
                
# df_joined = df_joined.copy().drop(['Num_obs', 'G',
#                                   'Critical_list_numbered_object_flag',  'Node', 'Peri', 'NEO_flag', 
#                                   'Num_opps', 'One_km_NEO_flag', 'One_opposition_object_flag', 'Orbital_period',
#                                   'PHA_flag', 'Semilatus_rectum', 'Synodic_period', 'rms',  'Tp','H'],
#                           axis=1)

df_joined = df_joined.copy().drop([ 'NEO_flag','PHA_flag'],
                          axis=1)

#substitute null values with 0s
df_joined = df_joined.fillna(0.0)
df_joined.dtypes

Aphelion_dist                         float64
Critical_list_numbered_object_flag    float64
Epoch                                 float64
G                                     float64
H                                     float64
M                                     float64
Node                                  float64
Num_obs                               float64
Num_opps                                int64
One_km_NEO_flag                       float64
One_opposition_object_flag            float64
Orbital_period                        float64
Peri                                  float64
Perihelion_dist                       float64
Semilatus_rectum                      float64
Synodic_period                        float64
Tp                                    float64
a                                     float64
e                                     float64
i                                     float64
n                                     float64
rms                               

In [11]:
# Extract the Label column as a Series from the master set
sr_label = df_joined['Label']

# Drop the Label column from the master data set
df_joined = df_joined.drop('Label', axis=1)

df_joined.columns

Index([u'Aphelion_dist', u'Critical_list_numbered_object_flag', u'Epoch', u'G',
       u'H', u'M', u'Node', u'Num_obs', u'Num_opps', u'One_km_NEO_flag',
       u'One_opposition_object_flag', u'Orbital_period', u'Peri',
       u'Perihelion_dist', u'Semilatus_rectum', u'Synodic_period', u'Tp', u'a',
       u'e', u'i', u'n', u'rms'],
      dtype='object')

In [12]:
# Make train and test sets
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_joined, sr_label, test_size=.25, random_state=0)

In [13]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(535187, 22)
(178396, 22)
(535187,)
(178396,)


In [14]:

# clf = svm.SVC(kernel='linear', C=1)
# scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=3, scoring="f1_weighted")
# fitted = clf.fit(X_train, y_train)
# clf.score(X_test, y_test)
# print scores


In [15]:
#-----  LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Linear Discriminant Analysis --- Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Linear Discriminant Analysis --- Normalized confusion matrix')
print(cm_normalized)

/Users/goris/anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Linear Discriminant Analysis --- Confusion matrix, without normalization
[[   234    155     22]
 [   731   2109    285]
 [   136    194 174530]]
Linear Discriminant Analysis --- Normalized confusion matrix
[[  5.69e-01   3.77e-01   5.35e-02]
 [  2.34e-01   6.75e-01   9.12e-02]
 [  7.78e-04   1.11e-03   9.98e-01]]


In [16]:
print('Linear DiscriminantAnalysis Score')
clf.score(X_test, y_test)

Linear DiscriminantAnalysis Score


0.99146281306755757

In [17]:
#-----  LogisticRegression

lr = linear_model.LogisticRegression(penalty='l1',dual=False,max_iter =100 )
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

y_pred = lr.predict(X_test)






In [18]:
print('Logistic Regression Score')
lr.score(X_test,y_test)

Logistic Regression Score


0.99584631942420232

In [19]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Logistic Regression --- Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Logistic Regression --- Normalized confusion matrix')
print(cm_normalized)


Logistic Regression --- Confusion matrix, without normalization
[[    23    386      2]
 [    14   2810    301]
 [     0     38 174822]]
Logistic Regression --- Normalized confusion matrix
[[  5.60e-02   9.39e-01   4.87e-03]
 [  4.48e-03   8.99e-01   9.63e-02]
 [  0.00e+00   2.17e-04   1.00e+00]]


In [20]:
#-----  Random Forest Classifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)

y_pred = rfc.predict(X_test)
print('Random FOrest Classifier Score')
rfc.score(X_test,y_test)


Random FOrest Classifier Score


0.99849211865736898

In [21]:
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Random FOrest Classifier --- Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Random FOrest Classifier --- Normalized confusion matrix')
print(cm_normalized)


Random FOrest Classifier --- Confusion matrix, without normalization
[[   189    222      0]
 [    46   3078      1]
 [     0      0 174860]]
Random FOrest Classifier --- Normalized confusion matrix
[[  4.60e-01   5.40e-01   0.00e+00]
 [  1.47e-02   9.85e-01   3.20e-04]
 [  0.00e+00   0.00e+00   1.00e+00]]


In [22]:
# # df_pha_test = df_pha.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
# #                             'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U',
# #                                   'Arc_length', 'Label','PHA_flag'],
# #                           axis=1).fillna(0.0)
# df_pha_test = df_pha.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
#                             'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U',
#                                   'Arc_length', 'Label'],
#                           axis=1).fillna(0.0)
# # df_pha_test = df_pha.copy().drop(['Aphelion_dist', 'H', 'Perihelion_dist', 'Num_obs', 'Epoch', 'G',
# #                                   'Critical_list_numbered_object_flag', 'M', 'Node', 'Peri', 'NEO_flag', 
# #                                   'Num_opps', 'One_km_NEO_flag', 'One_opposition_object_flag', 'Orbital_period',
# #                                   'PHA_flag', 'Semilatus_rectum', 'Synodic_period', 'rms', 'n', 'e', 'Tp', 'a'],
# #                           axis=1)
# clf.score(df_pha_test.head(500), np.zeros((500,), dtype=np.int))
# #clf.predict(df_pha_test.head(500))

In [23]:
# # df_nea_test = df_nea_pure.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
# #                             'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U',
# #                                   'Arc_length', 'Label','PHA_flag'],
# #                           axis=1).fillna(0.0)
# df_nea_test = df_nea_pure.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
#                             'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U',
#                                   'Arc_length', 'Label'],
#                           axis=1).fillna(0.0)
# # df_nea_test = df_nea_pure.copy().drop(['Aphelion_dist', 'H', 'Perihelion_dist', 'Num_obs', 'Epoch', 'G',
# #                                   'Critical_list_numbered_object_flag', 'M', 'Node', 'Peri', 'NEO_flag', 
# #                                   'Num_opps', 'One_km_NEO_flag', 'One_opposition_object_flag', 'Orbital_period',
# #                                   'PHA_flag', 'Semilatus_rectum', 'Synodic_period', 'rms', 'n', 'e', 'Tp', 'a'],
# #                           axis=1)
# clf.score(df_nea_test.head(500), np.ones((500,), dtype=np.int))

In [24]:
# # df_mp_test = df_mp_pure.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
# #                             'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U',
# #                                   'Arc_length', 'Label', 'PHA_flag'],
# #                           axis=1).fillna(0.0)
# df_mp_test = df_mp_pure.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
#                             'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U',
#                                   'Arc_length', 'Label', 'PHA_flag'],
#                           axis=1).fillna(0.0)
# # df_mp_test = df_mp_pure.copy().drop(['Aphelion_dist', 'H', 'Perihelion_dist', 'Num_obs', 'Epoch', 'G',
# #                                   'Critical_list_numbered_object_flag', 'M', 'Node', 'Peri', 'NEO_flag', 
# #                                   'Num_opps', 'One_km_NEO_flag', 'One_opposition_object_flag', 'Orbital_period',
# #                                   'PHA_flag', 'Semilatus_rectum', 'Synodic_period', 'rms', 'n', 'e', 'Tp', 'a'],
# #                           axis=1)
# arr = np.full((500, 1), 2, dtype=np.int)
# clf.score(df_mp_test.head(500), arr)